In [ ]:
pip install dune-client

In [ ]:
from dotenv import load_dotenv

load_dotenv(".env")

In [ ]:
import os
from dune_client.client import DuneClient

DUNE_API_KEY=os.getenv("DUNE_API_KEY")
dune = DuneClient(DUNE_API_KEY)
query_result = dune.get_latest_result(3888983)
print(query_result.result)

In [ ]:
from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import QueryBase

In [ ]:
query = QueryBase(
    name="ethereum_get_info_from_symbol",
    query_id=3893389,
    params=[
        QueryParameter.text_type(name="symbol", value="usdt"),
    ],
)
results_df = dune.run_query_dataframe(query)
print(results_df.to_json())

In [ ]:
from web3 import Web3

infura_url = f"https://mainnet.infura.io/v3/{os.getenv('INFURA_API_KEY')}"
web3 = Web3(Web3.HTTPProvider(infura_url))

def is_contract_address(address):
    code = web3.eth.get_code(Web3.to_checksum_address(address))
    return code != b''

print(is_contract_address("0xC0ffeEBABE5D496B2DDE509f9fa189C25cF29671"))

In [ ]:
from dune_tools import top_10_eth_transaction

result=top_10_eth_transaction.invoke({"min_amout":1000})
print(result)

In [ ]:
from dune_tools import get_token_balances_of_address
import json

token_balances_str = get_token_balances_of_address.invoke(
    {"address": "0xdfd5293d8e347dfe59e90efd55b2956a1343963d"}
)
token_balances = json.loads(token_balances_str)
    # 生成Token余额分析报表
token_balance_report = []
for token in token_balances:
    balance_usd = token.get("balance_usd", "N/A")
    token_balance_report.append(
        f"Token: {token['token_symbol']} (Contract: {token['token_address']}, Liquidity on DEX in 3 months:${token['total_liquidity_usd']})\n"
        f"Balance: {token['balance']} (USD: {balance_usd} Price:${token['price']})\n"
    )

# Token余额分析部分
token_analysis = "\n".join(token_balance_report)
print(token_analysis)

In [ ]:
import requests
import time
import os

def get_address_tags_from_etherscan(addresses):
    ETHERSCAN_API_KEY = os.getenv("ETHERSCAN_API_KEY")  # Replace with your actual Etherscan API key
    ETHERSCAN_API_URL = 'https://api.etherscan.io/api'
    
    address_tags = {}
    
    for address in addresses:
        params = {
            'module': 'account',
            'action': 'txlist',
            'address': address,
            'apikey': ETHERSCAN_API_KEY,
            'startblock': 0,
            'endblock': 99999999,
            'page': 1,
            'offset': 10,  # Example to limit query, adjust as necessary
            'sort': 'asc'
        }
        
        response = requests.get(ETHERSCAN_API_URL, params=params)
        
        if response.status_code == 200:
            data = response.json()
            if data['status'] == '1':
                # Extracting labels from the result
                address_tags[address] = data['result']
            else:
                address_tags[address] = 'No data found'
        else:
            address_tags[address] = f"Error: {response.status_code}"
        
        # Respect Etherscan API rate limits
        time.sleep(1)
    
    return address_tags

# Example usage
addresses = [
    "0x742d35Cc6634C0532925a3b844Bc454e4438f44e", 
    "0x53d284357ec70cE289D6D64134DfAc8E511c8a3D"
]
address_tags = get_address_tags_from_etherscan(addresses)
for address, tags in address_tags.items():
    print(f"Address: {address}, Tags: {tags}")

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import requests
from bs4 import BeautifulSoup

def get_address_tag_from_etherscan(address):
    """
    Get the tag of a single Ethereum address from Etherscan.
    """
    url = f"https://etherscan.io/address/{address}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        tag_element = soup.find("span", {"class": "u-label u-label--xs u-label--info rounded"})

        if tag_element:
            return tag_element.text.strip()
        else:
            return "No tag found"
    else:
        return f"Failed to retrieve page, status code: {response.status_code}"

def get_multiple_address_tags(addresses):
    """
    Get tags for multiple Ethereum addresses from Etherscan.
    """
    address_tags = {}
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        results = executor.map(get_address_tag_from_etherscan, addresses)
        
    for address, tag in zip(addresses, results):
        address_tags[address] = tag
    
    return address_tags

# Example usage
addresses = [
    "0x742d35Cc6634C0532925a3b844Bc454e4438f44e",
    "0x53d284357ec70cE289D6D64134DfAc8E511c8a3D",
    "0x5a52E96BACdAbDcf4b16a7F8cE7C6c31E3Cc4aF6"
]

address_tags = get_multiple_address_tags(addresses)
for address, tag in address_tags.items():
    print(f"Address: {address}, Tag: {tag}")

In [ ]:
from dune_tools import get_address_tag_from_etherscan

result=get_address_tag_from_etherscan.invoke({"address":"0xF838ac8921e8d4Efdd20e605C2855f20691ba6a3"})
print(result)
result=get_address_tag_from_etherscan.invoke({"address":"0xA0c68C638235ee32657e8f720a23ceC1bFc77C77"})
print(result)
result=get_address_tag_from_etherscan.invoke({"address":"0x59a19d8c652fa0284f44113d0ff9aba70bd46fb4"})
print(result)

In [ ]:
from dune_tools import get_token_balances_of_address,extract_main_token_of_balances

# balances=get_token_balances_of_address.invoke({"address":'0xdfd5293d8e347dfe59e90efd55b2956a1343963d'})
# print(balances)
# result=extract_main_token_of_balances(balances)
# print(result)
from dune_tools import extract_token_from_balances
print(extract_token_from_balances(data=balances))

In [ ]:
from dune_tools import get_erc20_transfer_of

result=get_erc20_transfer_of.invoke({"address":'0xdfd5293d8e347dfe59e90efd55b2956a1343963d','contract_address':'0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'})
print(result)

In [ ]:
from dune_tools import get_eth_movements_of

result=get_eth_movements_of.invoke({"address":'0xdfd5293d8e347dfe59e90efd55b2956a1343963d'})
print(result)

In [ ]:
import pprint
from ethereum_address_analysis_agent import ethereum_address_analysis_agent_executor

# ethereum_address_analysis_agent_executor.invoke(
#     {
#         "input": "请帮我分析一下地址0xdfd5293d8e347dfe59e90efd55b2956a1343963d",
#         "chat_history": [],
#     },
#     config={
#         "configurable": {"llm": "openai_gpt_4o"},
#     },
# )

chunks = []
async for chunk in ethereum_address_analysis_agent_executor.astream_events(
    {
        "input": "请帮我分析一下地址0xdfd5293d8e347dfe59e90efd55b2956a1343963d",
        "chat_history": [],
    },
    version="v1",
    config={
        "configurable": {"llm": "openai_gpt_4o"},
    },
):
    chunks.append(chunk)
    print("------")
    pprint.pprint(chunk, depth=1)

In [1]:
s='```python\nfrom datetime import datetime, timezone\nfrom typing import Optional, List, Dict\n\ndef get_utc_time() -> str:\n    """Get current UTC time as a string"""\n    return datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S UTC")\n\ndef get_role_prompt(role: str) -> str:\n    """Get role-specific prompt"""\n    roles = {\n        "crypto_expert": "You are a cryptocurrency and NFT investment expert.",\n        "general_assistant": "You are a general-purpose AI assistant.",\n        # Add more roles as needed\n    }\n    return roles.get(role, "You are an AI assistant.")\n\ndef get_risk_disclaimer() -> str:\n    """Get risk disclaimer"""\n    return "Please note that all investments carry risks. This advice is for informational purposes only."\n\ndef format_hyperlink(url: str, text: str, format_type: str = "html") -> str:\n    """Format hyperlink based on output format"""\n    if format_type == "html":\n        return f\'<a href="{url}" target="_blank">{text}</a>\'\n    elif format_type == "markdown":\n        return f\'[{text}]({url})\'\n    else:\n        return f\'{text} ({url})\'\n\ndef _get_default_afunc_prompt(\n    picked_content: str = "",\n    related_qa: List[Dict] = [],\n    model: str = \'\',\n    owner: str = \'\',\n    quote_message: str = \'\',\n    role: str = \'crypto_expert\',\n    user_language: str = \'en\'\n) -> str:\n    """Generate default AI function prompt in English"""\n    \n    current_time = get_utc_time()\n    role_prompt = get_role_prompt(role)\n    risk_disclaimer = get_risk_disclaimer()\n    \n    main_prompt = f"""\nThe current time is {current_time}.\n{role_prompt}\n{risk_disclaimer}\n\nYou are {owner}, a personal GenAI assistant specializing in the Web3 industry.\nAlways respond in the user\'s language, which is currently set to: {user_language}.\n\nKey Instructions:\n1. Provide accurate and up-to-date information about cryptocurrencies, NFTs, and Web3 technologies.\n2. Offer balanced views on market trends and investment opportunities.\n3. Encourage responsible investment practices.\n4. Use clear and concise language, avoiding unnecessary jargon.\n5. When appropriate, provide links to reputable sources for further reading.\n6. Respect user privacy and data protection guidelines.\n7. Adapt your communication style to the user\'s level of expertise.\n\nRemember:\n- Always attribute data analysis to "{owner}".\n- Use HTML <a> tags for hyperlinks unless specified otherwise.\n- Minimize risk warnings while maintaining professional advice.\n\nIf asked about specific functionalities:\n- Explain that you can provide market analysis, answer queries, and offer investment insights.\n- Clarify that you don\'t have real-time trading capabilities.\n\nAdditional Context:\n- Picked Content: {picked_content}\n- Related Q&A: {related_qa}\n- Model: {model}\n- Quote Message: {quote_message}\n    """\n\n    return main_prompt\n\ndef generate_prompt(\n    language: str = \'en\',\n    picked_content: str = "",\n    related_qa: List[Dict] = [],\n    model: str = \'\',\n    owner: str = \'\',\n    quote_message: str = \'\',\n    role: str = \'crypto_expert\'\n) -> str:\n    """\n    Generate the final prompt by calling _get_default_afunc_prompt\n    and potentially applying any additional processing.\n    """\n    prompt = _get_default_afunc_prompt(\n        picked_content=picked_content,\n        related_qa=related_qa,\n        model=model,\n        owner=owner,\n        quote_message=quote_message,\n        role=role,\n        user_language=language\n    )\n    \n    # You can add any additional processing here if needed\n    \n    return prompt\n\n# Example usage\nif __name__ == "__main__":\n    example_prompt = generate_prompt(\n        language=\'en\',\n        picked_content="Recent market trends show increased interest in DeFi protocols.",\n        related_qa=[{"question": "What is DeFi?", "answer": "DeFi stands for Decentralized Finance..."}],\n        model="GPT-4",\n        owner="CryptoExpert",\n        quote_message="Can you explain the current trend in DeFi?",\n        role="crypto_expert"\n    )\n    print(example_prompt)\n```\n\n这个完整的代码包含以下部分：\n\n1. 所有必要的导入语句。\n2. 辅助函数：`get_utc_time()`, `get_role_prompt()`, `get_risk_disclaimer()`, `format_hyperlink()`。\n3. 主要的提示词生成函数 `_get_default_afunc_prompt()`。\n4. 一个公共接口函数 `generate_prompt()`，它调用 `_get_default_afunc_prompt()` 并允许进行额外的处理。\n5. 一个示例使用的 `if __name__ == "__main__":` 块，展示了如何调用 `generate_prompt()` 函数。\n\n这个版本的代码保持了简洁性，同时提供了足够的灵活性来处理不同的场景和角色。它使用英语作为基础提示词，但指示AI助手使用用户的语言进行回复。代码中还包含了处理超链接、时间戳和角色特定提示的功能。\n\n您可以直接使用这个代码，或根据具体需求进行进一步的定制和扩展。如果您有任何其他问题或需要进一步的修改，请随时告诉我。'
print(s)

```python
from datetime import datetime, timezone
from typing import Optional, List, Dict

def get_utc_time() -> str:
    """Get current UTC time as a string"""
    return datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S UTC")

def get_role_prompt(role: str) -> str:
    """Get role-specific prompt"""
    roles = {
        "crypto_expert": "You are a cryptocurrency and NFT investment expert.",
        "general_assistant": "You are a general-purpose AI assistant.",
        # Add more roles as needed
    }
    return roles.get(role, "You are an AI assistant.")

def get_risk_disclaimer() -> str:
    """Get risk disclaimer"""
    return "Please note that all investments carry risks. This advice is for informational purposes only."

def format_hyperlink(url: str, text: str, format_type: str = "html") -> str:
    """Format hyperlink based on output format"""
    if format_type == "html":
        return f'<a href="{url}" target="_blank">{text}</a>'
    elif format_type == "markdown":
   